In [ ]:
from collections import namedtuple, defaultdict, Counter
from glob import glob
from itertools import groupby
from operator import itemgetter
from pathlib import Path
import pickle

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora import Dictionary
from gensim.models import LdaModel as LDA
from gensim.models import AuthorTopicModel as ATM

from matplotlib import pyplot as plt

import numpy as np

from pandas import DataFrame, read_csv, concat

import pyLDAvis as ldavis
import pyLDAvis.gensim

import seaborn as sns

from sklearn.model_selection import train_test_split

import scipy
from scipy.spatial.distance import cdist
from scipy.sparse import coo_matrix as sparse_matrix

ldavis.enable_notebook()
%matplotlib notebook
#%precision 4

out = Path('../output/')

Candidate = namedtuple('Candidate', ['iterations', 'num_topics'])



def get_i_t(filename):
    _, content, document_type = filename.split('-')
    i, t, _ = content.split('_')
    return int(i[1:]), int(t[1:]), document_type.split('.')[0]

get_texts = lambda df: df[target].str.split()
tobows = lambda df, d: concat([df['Anomaly_ID'], get_texts(df).apply(d.doc2bow)], axis=1)



isr2 = 2.0 ** -.5

def hellinger(x, y):
    return isr2 * np.sqrt(((np.sqrt(x) - np.sqrt(y)) ** 2).sum())



report_types = 'ISA', 'PFR', 'DPFR'

TEST_SIZE = 0.3
min_occurances = 2
target = 'GLOMUNSTEM'

%matplotlib notebook
%precision 4

BASEDIR = Path('../data')
OUT = Path('../output/')



In [ ]:
with open(BASEDIR / 'processed_authors.csv') as fd:
    af = read_csv(fd)
af.shape
af.head()

In [ ]:
authors_by_type = {
    t: af[af.ReportType == t] 
    for t in af.ReportType.unique()
}

In [ ]:
documents_by_type = {
    t: read_csv(OUT / f'norm_{t}.csv').dropna()
    for t in af.ReportType.unique()
}

In [ ]:
for document_type in documents_by_type:
    documents_by_type[document_type].Anomaly_ID = \
      'A' + documents_by_type[document_type].Anomaly_ID.apply(str)

    
    documents_by_type[document_type]['Attributed'] = False
    idx = documents_by_type[document_type].Anomaly_ID.isin(
        authors_by_type[document_type].Anomaly_ID.unique()
    )
    documents_by_type[document_type]['Attributed'][idx] = True
    
    idx = authors_by_type[document_type].Anomaly_ID.isin(
        documents_by_type[document_type].Anomaly_ID.unique()
    )
    authors_by_type[document_type] = authors_by_type[document_type][idx]


In [ ]:
def get_train_test_and_vocab(documents, test_size, min_occurances=min_occurances):
    train, test = train_test_split(
        documents, test_size=test_size,
        stratify=documents['Attributed']
    )
    train.reset_index(drop=True, inplace=True)
    test.reset_index(drop=True, inplace=True)

    vocab = Dictionary(
        train[target].str.split()
    )
    vocab.filter_extremes(no_below=min_occurances)

    return train, test, vocab


In [ ]:
with open('election_by_type.pkl', 'rb') as fd:
    election_by_type = pickle.load(fd)

model_by_type, train_documents_by_type, test_documents_by_type, dictionary_by_type = \
  dict(), dict(), dict(), dict()

train_author_table_by_type = dict()
test_author_table_by_type = dict()

def attribution_table(documents, relevent_authors):
    store = defaultdict(set)
    for idx, anomaly in documents.iterrows():
        authors_documents = relevent_authors[
            relevent_authors.Anomaly_ID == anomaly.Anomaly_ID
        ]

        for author in authors_documents.Users_ID:
            store[author].add(idx)

    return {k: list(v) for k, v in store.items()}


for document_type in election_by_type:
    c = election_by_type[document_type]

    train_documents_by_type[document_type], \
    test_documents_by_type[document_type], \
    dictionary_by_type[document_type] = \
        get_train_test_and_vocab(documents_by_type[document_type], TEST_SIZE)
    
    train_author_table_by_type[document_type] = attribution_table(
        train_documents_by_type[document_type],
        authors_by_type[document_type]
    )
    
    test_author_table_by_type[document_type] = attribution_table(
        test_documents_by_type[document_type],
        authors_by_type[document_type]
    )
    
    print(f'{document_type: <4}')


In [ ]:
for document_type in election_by_type:
    c = election_by_type[document_type]

    corpus = tobows(
        train_documents_by_type[document_type],
        dictionary_by_type[document_type]
    )[target]

    atm = ATM(
        corpus=list(corpus),
        author2doc=train_author_table_by_type[document_type],
        num_topics=c.num_topics,
        iterations=c.iterations,
    )
    model_by_type[document_type] = atm


In [ ]:
def get_document_topics(model, doc_bow):

    gamma_chunk, sstats = model.inference(
        chunk=[doc_bow], author2doc=dict(), doc2author=dict(), 
        rhot=1.00,
        collect_sstats=True
    )

    return gamma_chunk


def get_model_author_topic_vectors(model):
    author_topic_vectors = np.zeros(
        (model.num_authors, model.num_topics)
    )

    for i, author in enumerate(model.id2author.values()):
        idx, scores = zip(*model.get_author_topics(author))
        author_topic_vectors[i, idx] = scores

    return author_topic_vectors


def get_sorted_authors(model, doc_bow, author_topic_vectors, metric=hellinger):
    doc_vector = get_document_topics(model, doc_bow)

    author_scores = np.argsort(
        cdist(doc_vector, author_topic_vectors, metric=metric)
    )
    if author_scores:
        contenders = [
            model.id2author[idx]
            for idx in author_scores[0]
        ]
    else:
        contenders = []

    return contenders


def get_all_ranks_and_counts(model, publication_counts, authors, dictionary, documents):
    author_topic_vectors = get_model_author_topic_vectors(model)

    for idx, row in tobows(documents, dictionary).iterrows():
        Anomaly_ID, doc_bow = row['Anomaly_ID'], row[target]

        contenders = get_sorted_authors(model, doc_bow, author_topic_vectors)
        if not contenders:
            continue
        
        real_authors = [
            a
            for a in authors
            if idx in authors[a]
        ]

        for a in real_authors:
            try:
                rank = contenders.index(a)
            except ValueError as e:
                continue

            try:
                publications = publication_counts[a]
            except KeyError:
                continue

            yield publications, contenders.index(a)


In [ ]:
# train_author_table_by_type

publication_counts_by_type = dict()
for t in train_author_table_by_type:
    _authors = train_author_table_by_type[t]
    publication_counts_by_type[t] = {a: len(_authors[a]) for a in _authors}
    del _authors

train = list(get_all_ranks_and_counts(
    model_by_type['ISA'],
    publication_counts_by_type['ISA'],
    train_author_table_by_type['ISA'],
    dictionary_by_type['ISA'],
    train_documents_by_type['ISA']
))

test = list(get_all_ranks_and_counts(
    model_by_type['ISA'],
    publication_counts_by_type['ISA'],
    test_author_table_by_type['ISA'],
    dictionary_by_type['ISA'],
    test_documents_by_type['ISA']
))

In [ ]:
len(train), len(test)

In [ ]:
def plot_arc(arc):
    _ = plt.figure()
    # Load the dataset
    data = DataFrame(columns=["publications", "rank"], data=arc)
    _ = sns.jointplot(x=data['publications'], y=data['rank'], 
                      kind='hex',
                      #xlim=(0, 250), ylim=(0, 300), gridsize=20,
                     )

plot_arc(train)
plot_arc(test)